In [ ]:
# Import required libraries
import os
import time
from pathlib import Path
from typing import List, Tuple
import logging

# Jupyter-specific imports
from IPython.display import display, Image, HTML, clear_output
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual

# Import our screenshot tool
from livebook_screenshot_tool import LivebookScreenshotTool, LivebookScreenshotError

# Configure logging for Jupyter
logging.basicConfig(level=logging.INFO, format='%(levelname)s: %(message)s')

print("✅ All dependencies imported successfully!")
print("📚 Ready to capture Livebook screenshots!")


In [ ]:
def capture_and_display_screenshot(url: str, filename: str, headless: bool = True):
    """
    Capture a screenshot and display it in the notebook.
    """
    print(f"🚀 Starting screenshot capture...")
    print(f"📍 URL: {url}")
    print(f"📁 Filename: {filename}")
    print(f"👁️ Headless mode: {headless}")
    
    try:
        # Create the tool
        with LivebookScreenshotTool(headless=headless) as tool:
            # Capture screenshot
            success = tool.screenshot_livebook_toc(url, filename)
            
            if success:
                print("✅ Screenshot captured successfully!")
                
                # Find the screenshot file
                screenshot_path = Path("screenshots") / f"{filename}.png"
                if not screenshot_path.exists():
                    # Try with timestamp
                    screenshot_files = list(Path("screenshots").glob(f"{filename}*.png"))
                    if screenshot_files:
                        screenshot_path = screenshot_files[0]
                
                if screenshot_path.exists():
                    print(f"📸 Displaying screenshot: {screenshot_path}")
                    # Display the image in the notebook
                    display(Image(filename=str(screenshot_path), width=800))
                    
                    # Show file info
                    file_size = screenshot_path.stat().st_size
                    print(f"📊 File size: {file_size:,} bytes")
                else:
                    print("⚠️ Screenshot file not found for display")
            else:
                print("❌ Screenshot capture failed!")
                
    except LivebookScreenshotError as e:
        print(f"❌ Livebook Screenshot Error: {e}")
    except Exception as e:
        print(f"❌ Unexpected Error: {e}")

# Example usage
url = "https://klettbib.livebook.de/978-3-12-756141-8/"
capture_and_display_screenshot(url, "jupyter_basic_example", headless=True)


In [ ]:
def interactive_screenshot_tool(
    url="https://klettbib.livebook.de/978-3-12-756141-8/",
    filename="interactive_capture",
    headless=True,
    window_width=1920,
    window_height=1080
):
    """
    Interactive screenshot capture with configurable parameters.
    """
    print(f"🎛️ Interactive Screenshot Configuration")
    print("=" * 50)
    
    try:
        with LivebookScreenshotTool(
            headless=headless, 
            window_size=(window_width, window_height)
        ) as tool:
            
            success = tool.screenshot_livebook_toc(url, filename)
            
            if success:
                print("✅ Screenshot captured successfully!")
                
                # Display the screenshot
                screenshot_files = list(Path("screenshots").glob(f"{filename}*.png"))
                if screenshot_files:
                    latest_file = max(screenshot_files, key=lambda x: x.stat().st_mtime)
                    print(f"📸 Latest screenshot: {latest_file}")
                    display(Image(filename=str(latest_file), width=600))
                    
                    # File statistics
                    stats = latest_file.stat()
                    print(f"📊 File size: {stats.st_size:,} bytes")
                    print(f"🕒 Created: {time.ctime(stats.st_mtime)}")
            else:
                print("❌ Screenshot capture failed!")
                
    except Exception as e:
        print(f"❌ Error: {e}")

# Create interactive widget
interactive_widget = interactive(
    interactive_screenshot_tool,
    url=widgets.Text(
        value="https://klettbib.livebook.de/978-3-12-756141-8/",
        description="Livebook URL:",
        style={'description_width': 'initial'},
        layout=widgets.Layout(width='500px')
    ),
    filename=widgets.Text(
        value="interactive_capture",
        description="Filename:",
        style={'description_width': 'initial'}
    ),
    headless=widgets.Checkbox(
        value=True,
        description="Headless Mode",
        style={'description_width': 'initial'}
    ),
    window_width=widgets.IntSlider(
        value=1920,
        min=800,
        max=2560,
        step=100,
        description="Window Width:",
        style={'description_width': 'initial'}
    ),
    window_height=widgets.IntSlider(
        value=1080,
        min=600,
        max=1440,
        step=100,
        description="Window Height:",
        style={'description_width': 'initial'}
    )
)

display(interactive_widget)


In [ ]:
def process_isbn_batch(isbn_list: List[str], max_pages_per_book: int = 10):
    """
    Process a batch of ISBNs, capturing all pages for each book.
    Each book's screenshots are saved in a separate directory named after its ISBN.
    """
    print("📚 Starting ISBN Batch Processing")
    print("=" * 60)
    print(f"📋 Processing {len(isbn_list)} ISBNs")
    print(f"📄 Max pages per book: {max_pages_per_book}")
    print()
    
    # Create progress widgets
    overall_progress = widgets.IntProgress(
        value=0,
        min=0,
        max=len(isbn_list),
        description='Books:',
        bar_style='info',
        orientation='horizontal'
    )
    
    book_progress = widgets.IntProgress(
        value=0,
        min=0,
        max=max_pages_per_book,
        description='Pages:',
        bar_style='info',
        orientation='horizontal'
    )
    
    status_label = widgets.Label(value="Initializing...")
    
    display(overall_progress)
    display(book_progress)
    display(status_label)
    
    def progress_callback(book_index, total_books, isbn):
        """Update progress for current book."""
        overall_progress.value = book_index
        book_progress.value = 0  # Reset page progress
        status_label.value = f"📖 Processing {isbn} ({book_index + 1}/{total_books})"
    
    try:
        with LivebookScreenshotTool(headless=True) as tool:
            # Process all ISBNs
            results = tool.screenshot_isbn_batch(
                isbn_list=isbn_list,
                max_pages=max_pages_per_book,
                progress_callback=progress_callback
            )
            
            # Update final progress
            overall_progress.value = len(isbn_list)
            overall_progress.bar_style = 'success'
            book_progress.value = max_pages_per_book
            book_progress.bar_style = 'success'
            
            # Calculate summary statistics
            successful_books = sum(1 for result in results.values() if result['success'])
            total_pages = sum(result.get('total_pages', 0) for result in results.values() if result['success'])
            total_size = sum(result.get('total_size', 0) for result in results.values() if result['success'])
            
            status_label.value = f"🎉 Completed! {successful_books}/{len(isbn_list)} books, {total_pages} pages"
            
    except Exception as e:
        overall_progress.bar_style = 'danger'
        book_progress.bar_style = 'danger'
        status_label.value = f"❌ Batch processing failed: {e}"
        return None
    
    # Display detailed results
    print("\n📊 Detailed Results:")
    print("-" * 60)
    
    for isbn, result in results.items():
        if result['success']:
            pages = result['total_pages']
            size_mb = result['total_size'] / (1024 * 1024)
            print(f"✅ {isbn}: {pages:2d} pages, {size_mb:.1f} MB")
            print(f"   📁 Directory: screenshots/{isbn}/")
        else:
            print(f"❌ {isbn}: Failed - {result.get('error', 'Unknown error')}")
    
    print(f"\n📋 Summary:")
    print(f"   📚 Books processed: {len(isbn_list)}")
    print(f"   ✅ Successful: {successful_books}")
    print(f"   ❌ Failed: {len(isbn_list) - successful_books}")
    print(f"   📄 Total pages: {total_pages}")
    print(f"   💾 Total size: {total_size / (1024 * 1024):.1f} MB")
    
    return results

# Example ISBN list (replace with your actual ISBNs)
example_isbns = [
    "978-3-12-316302-9",
    # Add more ISBNs here
    # "978-3-12-123456-7",
    # "978-3-12-789012-3",
]

print("📋 Example ISBN List:")
for i, isbn in enumerate(example_isbns, 1):
    print(f"  {i}. {isbn}")

print(f"\n💡 To process these ISBNs, run:")
print("# results = process_isbn_batch(example_isbns, max_pages_per_book=10)")


In [ ]:
def filter_toc_with_ai(screenshots_dir="screenshots", openai_api_key=None):
    """
    Use AI to filter screenshots and identify TOC pages.
    
    Args:
        screenshots_dir: Directory containing ISBN subdirectories with screenshots
        openai_api_key: OpenAI API key (will try environment variable if not provided)
    """
    print("🤖 AI-Powered TOC Filtering")
    print("=" * 50)
    
    # Check for required dependencies
    try:
        from filter_toc_screenshots import TOCScreenshotFilter, TOCFilterError
    except ImportError:
        print("❌ TOC filter module not found!")
        print("💡 Make sure filter_toc_screenshots.py is in the same directory")
        return
    
    # Check for API key
    import os
    if not openai_api_key:
        openai_api_key = os.getenv('OPENAI_API_KEY')
    
    if not openai_api_key:
        print("⚠️ OpenAI API key required!")
        print("Set with: os.environ['OPENAI_API_KEY'] = 'your-key-here'")
        print("Or get one from: https://platform.openai.com/api-keys")
        return
    
    try:
        # Initialize the filter
        print("🔧 Initializing AI filter...")
        filter_tool = TOCScreenshotFilter(api_key=openai_api_key)
        
        # Check for screenshots
        if not os.path.exists(screenshots_dir):
            print(f"❌ Directory not found: {screenshots_dir}")
            return
        
        # Find ISBN directories
        isbn_dirs = [d for d in os.listdir(screenshots_dir) 
                    if os.path.isdir(os.path.join(screenshots_dir, d))]
        
        if not isbn_dirs:
            print("❌ No ISBN directories found")
            print("💡 Run the ISBN batch processing first")
            return
        
        print(f"📚 Found {len(isbn_dirs)} ISBN directories")
        
        # Progress widgets
        overall_progress = widgets.IntProgress(
            value=0,
            min=0,
            max=len(isbn_dirs),
            description='Books:',
            bar_style='info'
        )
        
        status_label = widgets.Label(value="Analyzing...")
        
        display(overall_progress)
        display(status_label)
        
        # Analyze all screenshots
        results = {}
        toc_pages_found = 0
        
        for i, isbn in enumerate(isbn_dirs):
            overall_progress.value = i
            status_label.value = f"📖 Analyzing {isbn}..."
            
            try:
                isbn_results = filter_tool.filter_isbn_directory(
                    os.path.join(screenshots_dir, isbn),
                    confidence_threshold=0.7
                )
                results[isbn] = isbn_results
                toc_pages_found += len(isbn_results['toc_pages'])
                
            except Exception as e:
                print(f"❌ Error analyzing {isbn}: {e}")
                results[isbn] = {'error': str(e)}
        
        # Update final progress
        overall_progress.value = len(isbn_dirs)
        overall_progress.bar_style = 'success'
        status_label.value = f"✅ Analysis complete! {toc_pages_found} TOC pages found"
        
        # Display results
        print("\\n📊 AI Analysis Results:")
        print("-" * 40)
        
        for isbn, book_results in results.items():
            if 'error' in book_results:
                print(f"❌ {isbn}: {book_results['error']}")
                continue
            
            toc_count = len(book_results['toc_pages'])
            total_count = book_results['total_screenshots']
            
            print(f"📖 {isbn}: {toc_count}/{total_count} pages are TOC")
            
            if toc_count > 0:
                for page in book_results['toc_pages']:
                    conf = page['confidence']
                    print(f"   ✅ {page['filename']} (confidence: {conf:.2f})")
        
        print(f"\\n🎯 Summary: {toc_pages_found} TOC pages identified across {len(isbn_dirs)} books")
        
        return results
        
    except Exception as e:
        print(f"❌ AI filtering failed: {e}")
        return None

# Example usage information
print("🤖 AI-Powered TOC Filtering")
print("This feature uses OpenAI GPT-4o mini to analyze screenshots and identify")
print("which ones contain table of contents.")
print()
print("📋 To use this feature:")
print("1. Set your OpenAI API key:")
print("   import os")
print("   os.environ['OPENAI_API_KEY'] = 'your-api-key-here'")
print()
print("2. Run the filtering:")
print("   # results = filter_toc_with_ai()")
print()
print("💡 This will analyze all screenshots and identify TOC pages automatically!")

# Show cost estimation
print("\\n💰 Cost Estimation:")
print("GPT-4o mini vision costs approximately $0.00015 per image")
print("For 10 pages per book: ~$0.0015 per book")
print("Very affordable for batch processing!")

# Uncomment to run with your API key:
# import os
# os.environ['OPENAI_API_KEY'] = 'your-api-key-here'
# results = filter_toc_with_ai()


In [ ]:
def filter_toc_with_ai(screenshots_dir="screenshots"):
    """
    Use AI to filter screenshots and identify TOC pages.
    Loads OpenAI API key from .env file automatically.
    """
    print("🤖 AI-Powered TOC Filtering")
    print("=" * 50)
    
    # Load environment variables from .env file
    try:
        from dotenv import load_dotenv
        load_dotenv()
        print("✅ Loaded environment variables from .env file")
    except ImportError:
        print("⚠️ python-dotenv not found. Install with: pip install python-dotenv")
        print("💡 You can still set OPENAI_API_KEY as a regular environment variable")
    
    # Check for required dependencies
    try:
        from filter_toc_screenshots import TOCScreenshotFilter, TOCFilterError
    except ImportError:
        print("❌ TOC filter module not found!")
        print("💡 Make sure filter_toc_screenshots.py is in the same directory")
        return
    
    # Check for API key
    import os
    openai_api_key = os.getenv('OPENAI_API_KEY')
    
    if not openai_api_key:
        print("⚠️ OpenAI API key not found!")
        print("🔑 Create a .env file in this directory with:")
        print("   OPENAI_API_KEY=your-api-key-here")
        print("💡 Get an API key from: https://platform.openai.com/api-keys")
        print("💰 Cost: ~$0.00015 per image (~$0.0015 per 10-page book)")
        return
    
    try:
        # Initialize the filter
        print("🔧 Initializing AI filter...")
        filter_tool = TOCScreenshotFilter()
        
        # Check for screenshots
        if not os.path.exists(screenshots_dir):
            print(f"❌ Directory not found: {screenshots_dir}")
            print("💡 Run the ISBN batch processing first")
            return
        
        # Find ISBN directories
        isbn_dirs = [d for d in os.listdir(screenshots_dir) 
                    if os.path.isdir(os.path.join(screenshots_dir, d)) and any(c.isdigit() for c in d)]
        
        if not isbn_dirs:
            print("❌ No ISBN directories found")
            print("💡 Run the ISBN batch processing first")
            return
        
        print(f"📚 Found {len(isbn_dirs)} ISBN directories")
        
        # Progress widgets
        overall_progress = widgets.IntProgress(
            value=0,
            min=0,
            max=len(isbn_dirs),
            description='Books:',
            bar_style='info'
        )
        
        status_label = widgets.Label(value="Analyzing...")
        
        display(overall_progress)
        display(status_label)
        
        # Analyze all screenshots
        results = {}
        toc_pages_found = 0
        total_cost = 0
        
        for i, isbn in enumerate(isbn_dirs):
            overall_progress.value = i
            status_label.value = f"📖 Analyzing {isbn}..."
            
            try:
                isbn_results = filter_tool.filter_isbn_directory(
                    os.path.join(screenshots_dir, isbn),
                    confidence_threshold=0.7
                )
                results[isbn] = isbn_results
                toc_pages_found += len(isbn_results['toc_pages'])
                
                # Estimate cost
                total_screenshots = isbn_results['total_screenshots']
                book_cost = total_screenshots * 0.00015
                total_cost += book_cost
                
            except Exception as e:
                print(f"❌ Error analyzing {isbn}: {e}")
                results[isbn] = {'error': str(e)}
        
        # Update final progress
        overall_progress.value = len(isbn_dirs)
        overall_progress.bar_style = 'success'
        status_label.value = f"✅ Analysis complete! {toc_pages_found} TOC pages found"
        
        # Display results
        print("\\n📊 AI Analysis Results:")
        print("-" * 40)
        
        for isbn, book_results in results.items():
            if 'error' in book_results:
                print(f"❌ {isbn}: {book_results['error']}")
                continue
            
            toc_count = len(book_results['toc_pages'])
            total_count = book_results['total_screenshots']
            
            print(f"📖 {isbn}: {toc_count}/{total_count} pages are TOC")
            
            if toc_count > 0:
                for page in book_results['toc_pages']:
                    conf = page['confidence']
                    print(f"   ✅ {page['filename']} (confidence: {conf:.2f})")
        
        print(f"\\n🎯 Summary:")
        print(f"   📚 Books analyzed: {len(isbn_dirs)}")
        print(f"   ✅ TOC pages found: {toc_pages_found}")
        print(f"   💰 Estimated cost: ${total_cost:.4f}")
        
        # Ask about organizing files
        organize = input("\\n🗂️ Organize files into toc_pages/ and non_toc_pages/ folders? (y/N): ").strip().lower()
        if organize == 'y':
            try:
                filter_tool.organize_filtered_results(results, organize_files=True)
                print("✅ Files organized!")
                print("📁 Check the toc_pages/ folders in each ISBN directory")
            except Exception as e:
                print(f"❌ Failed to organize files: {e}")
        
        # Save report
        filter_tool.save_analysis_report(results, "toc_analysis_report.json")
        print("💾 Detailed report saved: toc_analysis_report.json")
        
        return results
        
    except Exception as e:
        print(f"❌ AI filtering failed: {e}")
        return None

# Example usage information
print("🤖 AI-Powered TOC Filtering Setup")
print("=" * 40)
print("📋 Prerequisites:")
print("1. Create .env file with your OpenAI API key:")
print("   OPENAI_API_KEY=your-api-key-here")
print("2. Have screenshots from ISBN batch processing")
print("3. Install: pip install python-dotenv openai")
print()
print("🚀 Usage:")
print("# results = filter_toc_with_ai()")
print()
print("💰 Very affordable: ~$0.0015 per 10-page book!")

# Uncomment to run (after setting up .env file):
# results = filter_toc_with_ai()


In [ ]:
def batch_screenshot_with_progress(url_list: List[str], base_filename: str = "batch"):
    """
    Process multiple URLs with progress tracking and visualization.
    """
    print("🚀 Starting Batch Screenshot Processing")
    print("=" * 50)
    
    # Create progress widgets
    progress_bar = widgets.IntProgress(
        value=0,
        min=0,
        max=len(url_list),
        description='Progress:',
        bar_style='info',
        orientation='horizontal'
    )
    
    status_label = widgets.Label(value="Initializing...")
    
    display(progress_bar)
    display(status_label)
    
    results = {}
    successful_captures = []
    
    try:
        with LivebookScreenshotTool(headless=True) as tool:
            
            for i, url in enumerate(url_list):
                # Update progress
                progress_bar.value = i
                status_label.value = f"Processing {i+1}/{len(url_list)}: {url[:50]}..."
                
                filename = f"{base_filename}_{i+1:02d}"
                
                try:
                    success = tool.screenshot_livebook_toc(url, filename)
                    results[url] = success
                    
                    if success:
                        successful_captures.append(filename)
                        status_label.value = f"✅ Completed {i+1}/{len(url_list)}"
                    else:
                        status_label.value = f"❌ Failed {i+1}/{len(url_list)}"
                        
                except Exception as e:
                    results[url] = False
                    status_label.value = f"❌ Error {i+1}/{len(url_list)}: {str(e)[:30]}..."
                
                # Brief pause to show progress
                time.sleep(1)
            
            # Final update
            progress_bar.value = len(url_list)
            progress_bar.bar_style = 'success'
            
            successful_count = sum(results.values())
            status_label.value = f"🎯 Completed! {successful_count}/{len(url_list)} successful"
            
    except Exception as e:
        progress_bar.bar_style = 'danger'
        status_label.value = f"❌ Batch processing failed: {e}"
        return results
    
    # Display results summary
    print("\n📊 Batch Processing Results:")
    print("-" * 30)
    
    for url, success in results.items():
        status = "✅" if success else "❌"
        print(f"{status} {url}")
    
    print(f"\n🎯 Summary: {successful_count}/{len(results)} screenshots captured")
    
    # Display captured screenshots
    if successful_captures:
        print("\n📸 Captured Screenshots:")
        print("-" * 30)
        
        for filename in successful_captures[:3]:  # Show first 3
            screenshot_files = list(Path("screenshots").glob(f"{filename}*.png"))
            if screenshot_files:
                latest_file = max(screenshot_files, key=lambda x: x.stat().st_mtime)
                print(f"📷 {latest_file.name}")
                display(Image(filename=str(latest_file), width=400))
        
        if len(successful_captures) > 3:
            print(f"... and {len(successful_captures) - 3} more screenshots")
    
    return results

# Example batch processing
example_urls = [
    "https://klettbib.livebook.de/978-3-12-756141-8/",
    # Add more Livebook URLs here for real batch processing
    # "https://klettbib.livebook.de/another-book-id/",
]

print("📚 Example Batch Processing (modify urls list for real batch):")
# Uncomment the next line to run batch processing
# batch_results = batch_screenshot_with_progress(example_urls, "jupyter_batch")


In [ ]:
def analyze_livebook_page(url: str):
    """
    Analyze a Livebook page to understand its structure and TOC detection.
    """
    print("🔍 Analyzing Livebook Page Structure")
    print("=" * 50)
    print(f"📍 URL: {url}")
    
    try:
        with LivebookScreenshotTool(headless=True) as tool:
            info = tool.get_page_info(url)
            
            if 'error' in info:
                print(f"❌ Error analyzing page: {info['error']}")
                return
            
            # Display basic page info
            print(f"\n📄 Page Information:")
            print(f"  Title: {info['title']}")
            print(f"  URL: {info['url']}")
            print(f"  Page Source Length: {info['page_source_length']:,} characters")
            
            # Create visualization of TOC detection results
            print(f"\n🎯 TOC Element Detection Results:")
            print("-" * 60)
            
            found_selectors = []
            for selector, count in info['found_elements'].items():
                status = "✅" if count > 0 else "❌"
                print(f"{status} {selector:<50} | {count:>3} elements")
                
                if count > 0:
                    found_selectors.append((selector, count))
            
            # Summary
            total_found = len(found_selectors)
            print(f"\n📊 Detection Summary:")
            print(f"  Successful selectors: {total_found}/{len(info['found_elements'])}")
            
            if found_selectors:
                print(f"  Best candidates:")
                for selector, count in sorted(found_selectors, key=lambda x: x[1], reverse=True)[:3]:
                    print(f"    • {selector} ({count} elements)")
            else:
                print("  ⚠️ No TOC elements detected - page structure may be unusual")
                print("  💡 Try manual inspection or contact support")
    
    except Exception as e:
        print(f"❌ Analysis failed: {e}")

# Analyze the example Livebook page
analyze_livebook_page("https://klettbib.livebook.de/978-3-12-756141-8/")


In [ ]:
def list_captured_screenshots():
    """
    List and display all captured screenshots in the screenshots directory.
    """
    screenshots_dir = Path("screenshots")
    
    if not screenshots_dir.exists():
        print("📁 No screenshots directory found")
        return
    
    screenshot_files = list(screenshots_dir.glob("*.png"))
    
    if not screenshot_files:
        print("📸 No screenshots found in the directory")
        return
    
    print(f"📚 Found {len(screenshot_files)} screenshots:")
    print("=" * 50)
    
    # Sort by modification time (newest first)
    screenshot_files.sort(key=lambda x: x.stat().st_mtime, reverse=True)
    
    for i, file_path in enumerate(screenshot_files, 1):
        stats = file_path.stat()
        file_size = stats.st_size
        created_time = time.ctime(stats.st_mtime)
        
        print(f"{i}. {file_path.name}")
        print(f"   📊 Size: {file_size:,} bytes")
        print(f"   🕒 Created: {created_time}")
        
        # Display thumbnail
        display(Image(filename=str(file_path), width=300))
        print("-" * 30)

def cleanup_old_screenshots(keep_recent=5):
    """
    Clean up old screenshots, keeping only the most recent ones.
    """
    screenshots_dir = Path("screenshots")
    
    if not screenshots_dir.exists():
        print("📁 No screenshots directory found")
        return
    
    screenshot_files = list(screenshots_dir.glob("*.png"))
    
    if len(screenshot_files) <= keep_recent:
        print(f"📸 Only {len(screenshot_files)} screenshots found, no cleanup needed")
        return
    
    # Sort by modification time (oldest first)
    screenshot_files.sort(key=lambda x: x.stat().st_mtime)
    
    # Files to delete (all except the most recent ones)
    files_to_delete = screenshot_files[:-keep_recent]
    
    print(f"🧹 Cleaning up {len(files_to_delete)} old screenshots...")
    print(f"📁 Keeping {keep_recent} most recent screenshots")
    
    for file_path in files_to_delete:
        try:
            file_path.unlink()
            print(f"🗑️ Deleted: {file_path.name}")
        except Exception as e:
            print(f"❌ Failed to delete {file_path.name}: {e}")
    
    print("✅ Cleanup completed!")

# Quick utilities
print("🛠️ Utility Functions Available:")
print("• list_captured_screenshots() - View all captured screenshots")
print("• cleanup_old_screenshots(keep_recent=5) - Clean up old files")
print("\n📋 Example usage:")
print("# list_captured_screenshots()")
print("# cleanup_old_screenshots(keep_recent=3)")
